In [1]:
from pytorch_lightning import Trainer
from catinous.CatsinomModel import CatsinomModel
from catinous.CatsinomDataset import CatsinomDataset
import matplotlib.pyplot as plt
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.utils.data import DataLoader
import os
import torchvision.models as models
import torch.nn as nn
import torch
import torch.nn.functional as F
import visualizationutils as vu
import argparse
import pytorch_lightning.logging as pllogging
import pytorch_lightning as pl

import numpy as np

%load_ext autoreload
%autoreload 2

In [2]:
dsname = 'lr'

def trained_full_dataset(dsname = 'lr', base_only=False, continue_from=None):
    
    hparams = dict()
    hparams['root_dir'] = '/project/catinous/cat_data/'
    hparams['datasetfile'] = 'catsinom_' + dsname + '_dataset.csv'
    hparams['batch_size'] = 8
    hparams['train_iterations'] = 3000

    if base_only:
        hparams['trainsplit'] = ['base_train']
        dsname = dsname + 'base'
    else:
        hparams['trainsplit'] = ['train','base_train']
    
    model = CatsinomModel(argparse.Namespace(**hparams))

    if continue_from is not None:
        model.load_state_dict(torch.load('/project/catinous/trained_models/' + continue_from + '_iterations.pt'))
        outname = '/project/catinous/trained_models/' + continue_from + dsname + '_iterations.pt'
        dsname = dsname + continue_from
    else:
        outname = '/project/catinous/trained_models/' + dsname + '_iterations.pt'
        
    if not os.path.exists(outname):
        logger = pllogging.TestTubeLogger('catinous_log_iterations', name=dsname)
        trainer = Trainer(gpus=1, max_epochs=1, early_stop_callback=False, logger=logger, val_check_interval=500)
        trainer.fit(model)
        torch.save(model.state_dict(), outname)

    return outname

In [5]:
trained_full_dataset('hr')
trained_full_dataset('lr')
trained_full_dataset('combined')
trained_full_dataset('hr', base_only=True)
trained_full_dataset('lr', base_only=True)
trained_full_dataset('hr', continue_from='lr')
trained_full_dataset('lr', continue_from='hr')

INFO:root:gpu available: True, used: True
INFO:root:VISIBLE GPUS: 0
/home/mperkonigg/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/mperkonigg/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/mperkonigg/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/mperk

/project/catinous/cat_data/ catsinom_hr_dataset.csv


Epoch 1:   0%|          | 0/3318 [00:00<?, ?batch/s]

Epoch 1: 100%|██████████| 3318/3318 [14:14<00:00,  3.88batch/s, batch_idx=2999, gpu=0, loss=0.128, v_num=0]


INFO:root:gpu available: True, used: True
INFO:root:VISIBLE GPUS: 0
INFO:root:
              Name               Type Params
0            model             ResNet   24 M
1      model.conv1             Conv2d    9 K
2        model.bn1        BatchNorm2d  128  
3       model.relu               ReLU    0  
4    model.maxpool          MaxPool2d    0  
..             ...                ...    ...
150       model.fc         Sequential    1 M
151     model.fc.0             Linear    1 M
152     model.fc.1        BatchNorm1d    1 K
153     model.fc.2             Linear  513  
154           loss  BCEWithLogitsLoss    0  

[155 rows x 3 columns]
Validation sanity check:   0%|          | 0/5 [00:00<?, ?batch/s]

/project/catinous/cat_data/ catsinom_lr_dataset.csv


Epoch 1:   0%|          | 0/3288 [00:00<?, ?batch/s]

Epoch 1: 100%|██████████| 3288/3288 [13:40<00:00,  4.01batch/s, batch_idx=2999, gpu=0, loss=0.082, v_num=0]


'/project/catinous/trained_models/lrbase_iterations.pt'

In [5]:
trained_full_dataset('hr')

'/project/catinous/trained_models/hr_iterations.pt'

In [9]:
pllogging.__all__

['MLFlowLogger', 'TensorBoardLogger']

In [4]:
pllogging

<module 'pytorch_lightning.logging' from '/home/jhofmanninger/anaconda3/envs/candid/lib/python3.6/site-packages/pytorch_lightning/logging/__init__.py'>

In [29]:
np.random.seed(9375)
perm = np.random.permutation([1,2,3,4,5,6])
print(perm)

[4 2 1 3 5 6]
